In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Ubicación archivos
URL = 'C:/Users/juan.vergara/OneDrive - Caren Repuestos Flotacentro/Documentos/Análisis/Parque Automotriz/Parque en Bruto para Modelo/'

In [3]:
# Parque Automotriz
df_parque = pd.read_csv(URL+'ParqueAutomotriz2020.csv')

In [4]:
df_parque.shape[0]

7842085

In [5]:
df_parque.columns

Index(['ppu', 'dv_ppu', 'ano_fab', 'color', 'motor', 'chasis', 'rut', 'dv',
       'nomtot', 'tp_motor_fantasia', 'tp_marcaMotor', 'tp_vigencia', 'Marca',
       'Tipo', 'Modelo', 'Consumo_Teorico_Lubricantes',
       'Consumo_Teorico_Neumaticos', 'Consumo_Teorico_Repuestos',
       'Consumo_Teorico_Total', 'RUT_Final'],
      dtype='object')

In [6]:
# Después del 2000
df_parque = df_parque[df_parque['ano_fab'] >= 2000]

In [7]:
df_parque.shape[0]

6266808

In [8]:
# Borrando los NaN
df_parque.dropna(subset = ["Consumo_Teorico_Total"], inplace=True)

In [9]:
df_parque.shape[0]

4187299

In [10]:
df_parque.head(5)

,ppu,dv_ppu,ano_fab,color,motor,chasis,rut,dv,nomtot,tp_motor_fantasia,tp_marcaMotor,tp_vigencia,Marca,Tipo,Modelo,Consumo_Teorico_Lubricantes,Consumo_Teorico_Neumaticos,Consumo_Teorico_Repuestos,Consumo_Teorico_Total,RUT_Final
101,AC9708,1,2003,PLATEADO,W021780815,9BG138FW03C418739,8872397,K,CORTES ESCOBAR JOSE ERNESTO,NaN,NaN,NaN,CHEVROLET,CAMIONETA,S 10 D CAB 4X4,0.0,37500.0,0.0,37500.0,8872397.0
577,AB2385,6,2002,BLANCO,904925-508225,9BM693028-1B-277354,69251100,K,ILUSTRE MUNICIPALIDAD DE CAMINA,OM-904-LA / MERC. BENZ,MERCEDES BENZ,V,MERCEDES BENZ,CAMION,ATEGO 1315,416590.0,1000000.0,1500000.0,2916590.0,69251100.0
585,AB2393,7,2003,BLANCO,D4BF2-677335,KMJFD37FP3K-551078,6741524,8,ARCE FLORES CORNELIO,D4BF / HYUNDAI,HYUNDAI,V,HYUNDAI,MINIBUS,GRACE H 100,120000.0,200000.0,400000.0,720000.0,6741524.0
588,AB2396,1,2004,VERDE,YD25110790A,JN1CPGD22U0760654,69251100,K,ILUSTRE MUNICIPALIDAD DE CAMINA,NaN,NaN,NaN,NISSAN,CAMIONETA,TERRANO DX D CAB 2.5,0.0,37500.0,0.0,37500.0,69251100.0
591,AB2399,6,2006,NARANJO,904957638976,9BM6882765B431151,69251100,K,ILUSTRE MUNICIPALIDAD DE CAMINA,OM-904-LA / MERC. BENZ,MERCEDES BENZ,V,MERCEDES BENZ,LOC. COLECTIVA,LO 915,208295.0,511467.0,900000.0,1619762.0,69251100.0


In [11]:
# Clasificación tipo de vehículo
df_tipo = pd.read_excel(URL+'Dim_TipoVehiculo.xlsx', sheet_name = 'Hoja1')
df_tipo.columns = ['Tipo','tipo_corto','clasificacion']
df_tipo.head()

,Tipo,tipo_corto,clasificacion
0,AMBULANCIA,OTRO,OTRO
1,AUTOMOVIL,AUTOMOVIL,LIVIANO
2,BICICLETA MOTOR,OTRO,OTRO
3,BICIMOTO,OTRO,OTRO
4,BUGGI,OTRO,OTRO


In [12]:
df_parque = pd.merge(df_parque, df_tipo, on = 'Tipo', how = 'left')

In [13]:
# Traducción Pesados
df_parque.loc[(df_parque['tp_vigencia'] != 'V') & (df_parque['clasificacion'] == 'PESADO'), 'Consumo_Teorico_Neumaticos'] = 0

In [14]:
# Base a trabajar

df_basetrabajo = pd.read_excel(URL + 'Listado de modelos a trabajar.xlsx', sheet_name = 'BD')
df_basetrabajo = df_basetrabajo[['llave_basetrabajo_pesados','vigencia']]
df_basetrabajo

,llave_basetrabajo_pesados,vigencia
0,AGRALE MA,V
1,FORD CARGO 915,V
2,FORD CARGO 1722,V
3,FORD CARGO 1723,V
4,FORD CARGO 1717,V
...,...,...
97,VOLVO FMX,V
98,VOLVO FM 11,V
99,VOLVO FM 13,V
100,VOLVO B420,V


In [15]:
# Left join

df_parque['llave_basetrabajo_pesados'] = df_parque['Marca']+' '+df_parque['Modelo']

df_parque = pd.merge(df_parque, df_basetrabajo, on = 'llave_basetrabajo_pesados', how = 'left')

In [16]:
df_parque.loc[(df_parque['vigencia'] != 'V'), 'Consumo_Teorico_Repuestos'] = 0

In [17]:
# Recálculo Consumo Teórico Total
df_parque.drop('Consumo_Teorico_Total',1)

df_parque['Consumo_Teorico_Total'] = df_parque['Consumo_Teorico_Lubricantes'] + df_parque['Consumo_Teorico_Repuestos'] + df_parque['Consumo_Teorico_Neumaticos']



In [18]:
# Solo Pesados
df_parque = df_parque[df_parque.clasificacion == 'PESADO']

In [19]:
df_parque.shape[0]

479563

In [20]:
# Consumo 
df_consumoporrut = df_parque.groupby(['RUT_Final']).agg(Consumo_Teorico_Total = ('Consumo_Teorico_Total','sum'), 
                                                        Consumo_Teorico_Lubricantes = ('Consumo_Teorico_Lubricantes','sum'),
                                                       Consumo_Teorico_Repuestos = ('Consumo_Teorico_Repuestos','sum'),
                                                       Consumo_Teorico_Neumaticos = ('Consumo_Teorico_Neumaticos','sum')).reset_index()
df_consumoporrut.head()

,RUT_Final,Consumo_Teorico_Total,Consumo_Teorico_Lubricantes,Consumo_Teorico_Repuestos,Consumo_Teorico_Neumaticos
0,0.0,664213204.0,128002250.0,243100000.0,293110954.0
1,158189.0,320000.0,120000.0,0.0,200000.0
2,165920.0,320000.0,120000.0,0.0,200000.0
3,552734.0,4333180.0,833180.0,1500000.0,2000000.0
4,586078.0,11666360.0,1666360.0,6000000.0,4000000.0


In [21]:
# condiciones

condiciones = [(df_consumoporrut['Consumo_Teorico_Total'] < 10000000),
              (df_consumoporrut['Consumo_Teorico_Total'] >= 10000000) & (df_consumoporrut['Consumo_Teorico_Total'] < 40000000),
              (df_consumoporrut['Consumo_Teorico_Total'] >= 40000000) & (df_consumoporrut['Consumo_Teorico_Total'] < 200000000),
              (df_consumoporrut['Consumo_Teorico_Total'] >= 200000000) & (df_consumoporrut['Consumo_Teorico_Total'] < 60000000),
              (df_consumoporrut['Consumo_Teorico_Total'] >= 600000000) & (df_consumoporrut['Consumo_Teorico_Total'] < 1000000000),
              (df_consumoporrut['Consumo_Teorico_Total'] >= 1000000000)]

valores = ['1. [ , 10MM)','2. [10MM,40MM)','3. [40MM,200MM)','4. [200MM,600MM)','5. [600MM,1000MM)','6. [1000MM,)']

df_consumoporrut['Categoria_Potencial'] = np.select(condiciones, valores)

In [22]:
df_consumoporrut.to_csv(URL+'potencial_pesados_por_RUT.csv', index = False)

In [23]:
print('$', df_consumoporrut['Consumo_Teorico_Neumaticos'].sum())
print('US',df_consumoporrut['Consumo_Teorico_Neumaticos'].sum()/800)

$ 299485163835.0
US 374356454.79375


In [24]:
print('$', df_consumoporrut['Consumo_Teorico_Repuestos'].sum())
print('US',df_consumoporrut['Consumo_Teorico_Repuestos'].sum()/800)

$ 219297600000.0
US 274122000.0


In [25]:
print('$', df_consumoporrut['Consumo_Teorico_Lubricantes'].sum())
print('US',df_consumoporrut['Consumo_Teorico_Lubricantes'].sum()/800)

$ 130706450410.0
US 163383063.0125


In [26]:
print('$', df_consumoporrut['Consumo_Teorico_Total'].sum())
print('US',df_consumoporrut['Consumo_Teorico_Total'].sum()/800)

$ 649489214245.0
US 811861517.80625


In [27]:
df_consumoporrut = df_consumoporrut.groupby(['Categoria_Potencial']).agg(Num_RUT = ('RUT_Final', 'count')).reset_index()

In [28]:
df_consumoporrut

,Categoria_Potencial,Num_RUT
0,0,88
1,"1. [ , 10MM)",164687
2,"2. [10MM,40MM)",6777
3,"3. [40MM,200MM)",974
4,"5. [600MM,1000MM)",22
5,"6. [1000MM,)",31


In [ ]:
#sns.set_style("whitegrid") 
#bar, ax = plt.subplots(figsize=(10,6))

#ax = sns.barplot(x = 'Categoría_Potencial', y = 'Num_RUT', data = df_consumoporrut)

#ax.set_title("Pie chart approximation in Seaborn - Total Tip by Delivery Type", fontsize = 15)
#ax.set_xlabel ("Delivery Type")
#ax.set_ylabel ("Percentage")